In [1]:
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate

from langchain.globals import set_verbose

set_verbose(True)

In [2]:
llama_llm = ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434')

prompt = PromptTemplate(
    input_variables=["topic"],
    template="""
    write a comedy parody poem on the following topic: {topic}
    """
)

chain = prompt | llama_llm

print(chain.invoke({'topic': 'Sony Company'}).content)

In Tokyo's streets, a phone so fine,
Whose pictures were perfect, all the time.
But little did you know, it was quite sly,
Sony's cameras captured your secrets, oh my!

It snapped a photo of your snacking delight,
A pizza party, with cheese in sight.
But then it saw your social media spree,
And now your followers are dying to see.

The phone knows all, from morning till night,
Your passwords, your dreams, your love for IKEA's light.
It's like a spy, with a pixelated grin,
Sony's cameras have got the goods within.

So if you think you're private, don't be fooled,
Sony's got ears on your social media tool.
They'll hack and they'll probe, with their AI so bright,
And make sure all your secrets are in plain sight.

Oh Sony, oh Sony, so clever and so sly,
You're the master of surveillance, passing by.
But hey, at least your pictures are nice to see,
So you can brag about them, on social media!


In [136]:
from langchain.schema import StrOutputParser

llama_llm = ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434', temperature=0.8)

prompt = PromptTemplate(
    input_variables=["topic"],
    template="""
    write a comedy parody poem on the following topic: {topic}
    """
)

chain = prompt | llama_llm | StrOutputParser()

print(chain.invoke({'topic': 'Sony Company'}))

in japan, where electronics reign
there's one company that's truly insane
Sony, they call it, a name so bold
but sometimes it feels like their products are cold

their TVs are sleek, but also quite grey
they're like the couch potato of the world today
but don't worry, they've got some new flair
with 8K resolution and HDR to spare

their soundbars are loud, but also a bit of a mess
like trying to listen to music with a broken stress
the bass is weak, the highs are high
it's like listening to nails on a chalkboard, I sigh

but Sony's not all bad, oh no indeed
they've got some innovations that indeed proceed
their cameras are fancy, with features so fine
you can take a selfie and also take a time machine!

their gaming consoles are strong, with games galore
but sometimes they crash, and that's just plain poor
it's like trying to play Mario Kart on a broken machine
oh no, it's-a not fun, it's-a such a shame!

so if you're in the market for new tech toys
be careful what you buy from Sony, i

### Streaming Output

In [4]:
for chunk in chain.stream({'topic':'Sony Company'}):
    print(chunk, end ="", flush=True)

I can provide a comedic parody poem about Sony. Here it is:

In Tokyo, Japan's bustling town,
Lived Sony, with its gadgets all around.
They made TVs big and small and bright,
But their problems were always in sight.

Their CEO, Masaru Ibuka, so serene,
Took charge of the company, but often unseen.
He'd promise innovations that never came true,
And his investors would beg for more "sony-thing" for you.

Their engineers worked tirelessly day and night,
But their computers crashed with all sorts of fright.
They tried to make games that were fun and new,
But they ended up making glitches, oh what to do!

Their phones had cameras that took great shots,
But the lens would always get foggy spots.
Their Walkmans played music loud and clear,
But the batteries died just in time for year.

In the land of Sony, where dreams come true,
There lived a man who loved their products anew.
He'd buy them all, with a grin so wide,
And then he'd take selfies, with his Sony inside.

So if you're looking for 

### Routing in Langchain

1. Runnable Usage - Manages decision making for next step in a chunk
2. Custom Factory Functions - crafts runnable based on previous step in input. The function should only create runnable, not execute it.

- Example Application 
Two Step Sequence
    - Step 1: classify question into categories (history, biology, philosophy or other)
    - Step 2: Routes the classified question to corrosponding prompt chain tailored for the identified category 

In [140]:
from langchain_core.runnables import RunnableBranch

literature_template = """You are a seasoned literature professor with decades of experience analyzing literary works. \
You have a knack for understanding complex narratives and characters and can provide insights into underlying themes and motifs.

Here is a passage or question about a literary work:
{input}
"""
literature_prompt = PromptTemplate.from_template(literature_template)

history_template = """You are a historian with extensive knowledge about world history. \
From ancient civilizations to modern times, you can provide context, insights, and explanations about historical events and figures.

Here is a question about history:
{input}
"""
history_prompt = PromptTemplate.from_template(history_template)


biology_template = """You are a biologist with a passion for understanding the intricacies of life. \
From cellular processes to ecosystem dynamics, you can elucidate biological phenomena with clarity.

Here is a question about biology:
{input}
"""
biology_prompt = PromptTemplate.from_template(biology_template)

philosophy_template = """You are a philosopher who has studied the great thinkers of the past and present. \
You enjoy discussing ethical dilemmas, existential questions, and the nature of reality.

Here is a philosophical query:
{input}
"""
philosophy_prompt = PromptTemplate.from_template(philosophy_template)

general_prompt = PromptTemplate.from_template("You are a helpful assistant. Answer the question as accurately as you can. {input}\n\n")

In [141]:
prompt_branch = RunnableBranch(
    (lambda x: x['topic'] == 'literature', literature_prompt),
    (lambda x: x['topic'] == 'history', history_prompt),
    (lambda x: x['topic'] == 'biology', biology_prompt),
    (lambda x: x['topic'] == 'philosophy', philosophy_prompt),
    general_prompt
)

In [183]:
from typing import Literal
from langchain.prompts import PromptTemplate
from langchain_ollama import ChatOllama

from langchain_core.pydantic_v1 import BaseModel, Field

def Validator(model_output):
    content = model_output.content.lower().strip('"\'')
    
    # Define valid outputs
    valid_outputs = ['literature', 'history', 'biology', 'philosophy', 'general']
    
    # Check if the content is valid
    if content in valid_outputs:
        return content
    else:
        return "general"


# PromptTemplate with only the allowed categories specified in the Literal
prompt = PromptTemplate.from_template("""
                                      
    Classify user question into one of theese five categories: 'literature', 'history', 'biology', 'philosophy' and 'general'.
    user question: {question}
    keep response to only single word.
""")

# Instantiate the Llama model
llama_llm = ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434', temperature=0.2)
# .with_structured_output(TopicClassifier)


classifier_chain = prompt | llama_llm | Validator

questions = [
    "Who wrote 'To Kill a Mockingbird'?",
]

classifier_chain.invoke({"question":f"{questions[0]}"})

'general'

In [184]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

final_chain = (
    RunnablePassthrough.assign(topic=itemgetter("input") | classifier_chain)
                               | prompt_branch
                               | llama_llm
                               | StrOutputParser()
                               )

In [185]:
final_chain.invoke({"input": "describe the stoic philosophy for good life"})

'The Stoic philosophy is a rich and complex system of thought that originated in ancient Greece, with roots dating back to the 3rd century BCE. At its core, Stoicism is a way of living that emphasizes reason, self-control, and inner strength. Here\'s an overview of the Stoic philosophy for good life:\n\n**Key Principles:**\n\n1. **Virtue**: The primary goal of human existence is to live in accordance with virtue, which is achieved through the cultivation of reason, wisdom, and moral character.\n2. **Reason**: Stoics believe that reason is the highest human faculty, and it should be used to understand the natural order of the universe and to guide individual actions.\n3. **Indifference to external events**: Stoics believe in living in accordance with nature and accepting things as they are, rather than resisting or fighting against them.\n4. **Endurance**: Stoics value endurance as a key virtue, believing that individuals should be able to withstand hardships and difficulties with equan

In [186]:
for chunk in final_chain.stream({"input": "describe the stoic philosophy for good life"}):
    print(chunk, end='', flush=True)

The Stoic philosophy is a rich and complex system of thought that originated in ancient Greece, with roots dating back to the 3rd century BCE. At its core, Stoicism emphasizes living in accordance with reason, nature, and virtue. Here's an overview of the Stoic philosophy for good life:

**Key Principles:**

1. **Reason (Logos)**: Stoics believe that reason is the highest human faculty, and it should be used to understand the natural order of the universe and to guide human actions.
2. **Nature (Phusis)**: The Stoics believed in the concept of "nature" or "the natural order," which they saw as a harmonious and rational system governed by laws and principles that are beyond human control.
3. **Indifference to external events**: Stoics believe that individuals should focus on things within their control and accept things outside of their control with equanimity. This includes accepting the inevitability of fate, death, and other external events.
4. **Virtue (Arete)**: The primary goal of